In [ ]:
#pip install flask

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importa as bibliotecas necessárias
from flask import Flask, render_template, request, jsonify
import sqlite3

# Cria uma instância do aplicativo Flask
aplicativo = Flask(__name__)

In [ ]:
# Dados do cardápio (poderia vir de um banco de dados)
cardapio = {
    1: {"nome": "Hambúrguer Clássico", "preco": 25.00},
    2: {"nome": "Pizza Margherita", "preco": 35.00},
    3: {"nome": "Salada Caesar", "preco": 20.00},
    4: {"nome": "Batata Frita", "preco": 12.00},
    5: {"nome": "Refrigerante", "preco": 5.00}
}

# Promoções atuais
promocoes = ["Na compra de 2 hambúrgueres, ganhe uma porção de batata frita!", "Pizza Margherita com 10% de desconto hoje!"]

# Histórico de conversas (para manter o contexto, opcional para este exemplo básico)
historico_conversas = {}

def responder(mensagem_usuario, id_usuario="guest"):
    """Função principal para gerar a resposta do chatbot."""
    mensagem_usuario = mensagem_usuario.lower()

    if id_usuario not in historico_conversas:
        historico_conversas[id_usuario] = []

    historico_conversas[id_usuario].append({"usuario": mensagem_usuario})

    if "cardápio" in mensagem_usuario:
        resposta = "Nosso cardápio:\n"
        for numero, item in cardapio.items():
            resposta += f"{numero}. {item['nome']} - R$ {item['preco']:.2f}\n"
    elif "pedir" in mensagem_usuario:
        resposta = "Para fazer um pedido, digite o número dos itens desejados separados por vírgula (ex: 1, 3)."
    elif any(str(numero) in mensagem_usuario for numero in cardapio):
        itens_pedido = []
        numeros_digitados = [int(s) for s in mensagem_usuario.split() if s.isdigit()]
        for numero in numeros_digitados:
            if numero in cardapio:
                itens_pedido.append(cardapio[numero]['nome'])
        if itens_pedido:
            resposta = f"Você adicionou ao seu pedido: {', '.join(itens_pedido)}. Algo mais?"
        else:
            resposta = "Desculpe, não entendi os itens do seu pedido. Por favor, digite os números corretamente."
    elif "promoções" in mensagem_usuario:
        if promocoes:
            resposta = "Nossas promoções de hoje são:\n" + "\n".join(f"- {promo}" for promo in promocoes)
        else:
            resposta = "No momento, não temos promoções."
    elif "olá" in mensagem_usuario or "oi" in mensagem_usuario or "bom dia" in mensagem_usuario or "boa tarde" in mensagem_usuario or "boa noite" in mensagem_usuario:
        resposta = "Olá! Bem-vindo ao nosso restaurante. Como posso ajudar hoje?\n\nDigite o número da opção desejada:\n1. Ver Cardápio\n2. Fazer um Pedido\n3. Ver Promoções"
    else:
        resposta = "Desculpe, não entendi sua pergunta. Posso te ajudar com o cardápio, fazer um pedido ou informar sobre as promoções."

    historico_conversas[id_usuario].append({"chatbot": resposta})
    return resposta

@app.route("/")
def index():
    """Rota principal para exibir a página web com o chatbot."""
    return render_template("index.html")

@app.route("/chatbot", methods=["POST"])
def chatbot():
    """Rota para receber a mensagem do usuário e retornar a resposta do chatbot."""
    mensagem = request.form["mensagem"]
    resposta = responder(mensagem)
    return jsonify({"resposta": resposta})

if __name__ == "__main__":
    app.run(debug=True)